In [3]:
def self():
  self.integrator = 0
  self.preverr = 0
  self.differentiator = 0
  self.prevcx = 0

In [ ]:

def correct_error(self.prevcx,self.preverr ,cx, cy, image):
  maxlimit = 0     # this is maximum and minimum limits of system (in floats)
  minlimit = 0

  T = 10.00        # this is sample time basically to change from continuous to discrete time
  tau = 100.00     # time constant

  h, w, d = image.shape
  # NOTE: If it doesn't seems to work, try changing to cy to cx
  err = cx - w/2
  # NOTE: If the motion is too erratic, then consider a median range
  #      where there shall be no lateral movement
  
  # TODO: Propel linearly
  linear_thrust = 1600
  Kp = 0.4
  rot_thrustP = err*Kp

  # Integral
  Ki = 0.3
  self.integrator = self.integrator + 0.5*Ki*(err + self.preverr)

  # Anti-wind up via dynamic integrator clamping
  minInteg =  100.00
  maxInteg = 500.00

  # finding dynamic limits for integrator
  if maxlimit < rot_thrustP:
    maxInteg = maxlimit - rot_thrustP
  else:
    maxInteg = float(0)

  if minlimit < rot_thrustP:
    minInteg = minlimit - rot_thrustP
  else:
    minInteg = float(0)
  
  #Clamping the limits onto integrator
  if self.integrator > maxInteg:
    self.integrator = maxInteg
  elif self.integrator < minInteg:
    self.integrator = minInteg

  #differentiator
  Kd = 0.3          #note Kd has a additional sign -ve since we wanna avoid kick at setpoint change.
  self.differentiator = (2.0*-Kd*(cx-self.prevcx)+(((2.0)*tau-T)*self.differentiator))/(2.0*tau+T)

  # summing up and output
  rot_thrust = rot_thrustP + self.differentiator + self.integrator

  # clamping according to the limits
  # if rot_thrust > maxlimit:
  #   rot_thrust = maxlimit
  # elif rot_thrust < minlimit:
  #   rot_thrust = minlimit
  
  # final out put of the PID controller is rot_thrust
  
  rot_thrust = max(-100, min(rot_thrust, 100)) #probably clamper!!!
  if rot_thrust >= 0:
      print('Left')
  else:
      print('Right')

  print('rot_thrust = ', rot_thrust)

  err_y = cy - h/2
  constant = 90
  y_slope = 1/30.
  m.hp_control(err_y*y_slope + constant)

  thrusts = {
      m.pin_l: linear_thrust + rot_thrust,
      m.pin_r: linear_thrust - rot_thrust,
    }
  m.custom_thrusts(thrusts)

  return err, self.integrator, self.differentiator, cx
